In [1]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasisLong5.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
age = []
fc = []
subs = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for subtask in basis:
    m = re.search('([^-]+)-(.*)', subtask)
#     m = re.search('([^-]+)', subtask)
    sub = m.group(1)
    task = m.group(2)
    a = demo['age_at_cnb'][sub]
    age.append(a)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    fc.append(p)
    subs.append(sub)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
age = np.array(age)
fc = np.stack(fc)

print([a.shape for a in [thetas, jitter, age, fc]])
print(np.mean(age))

[(3849, 5, 264), (3849, 5, 264), (3849,), (3849, 34716)]
14.398285268901013


In [8]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps
    
ps = tops(thetas, jitter)
aps = np.mean(ps, axis=1)
res = fc - aps

print(ps.shape)
print(aps.shape)
print(res.shape)

(3849, 5, 34716)
(3849, 34716)
(3849, 34716)


In [9]:
res5 = res
aps5 = aps

print('Done')

Done


In [144]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from pathlib import Path
import re

def get_next_idx(model, inp):
    mx = -1
    for f in Path(f'{demodir}/weights/{inp}/').iterdir():
        m = re.match(f'{model}([\d]+).pkl', f.name)
        if m:
            idx = int(m.group(1))
            if idx > mx:
                mx = idx
    return mx+1

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

def save(model, inp, rmse, trsubs, tsubs, w):
    num = get_next_idx(model, inp)
    desc = f'{model} {inp} rmse: {rmse}'
    dct = dict(w=w.squeeze(), trsubs=trsubs, tsubs=tsubs, desc=desc)
    with open(f'{demodir}/weights/{inp}/{model}{num}.pkl', 'wb') as f:
        pickle.dump(dct, f)

idcs = np.random.permutation(fc.shape[0])
x0 = aps5
x1 = res1
x2 = fc
y = age
idcs = np.arange(fc.shape[0])

xtr0, xt0, xtr1, xt1, xtr2, xt2, tridcs, tidcs, ytr, yt = train_test_split(
    x0, x1, x2, idcs, y, train_size=0.5)

trsubs = [subs[i] for i in tridcs]
tsubs = [subs[i] for i in tidcs]

mu = np.mean(ytr)
ytr -= mu
yt -= mu

# reg0 = Ridge(alpha=1).fit(xtr0, ytr)
# yhat0 = reg0.predict(xt0)
# err = rmse(yhat0, yt)
# print(err)
# save('ridge', 'ab5-age', err, trsubs, tsubs, reg0.coef_)
# save('ridge-wf', 'ab5-age', err, trsubs, tsubs, reg0.coef_*np.mean(xtr0, axis=0))

# reg1 = Ridge(alpha=1).fit(xtr1, ytr)
# yhat1 = reg1.predict(xt1)
# err = rmse(yhat1, yt)
# print(err)
# save('ridge', 'res1-age', err, trsubs, tsubs, reg1.coef_)
# save('ridge-wf', 'res1-age', err, trsubs, tsubs, reg1.coef_*np.mean(xtr1, axis=0))

# reg2 = Ridge(alpha=1).fit(xtr2, ytr)
# yhat2 = reg2.predict(xt2)
# err = rmse(yhat2, yt)
# print(err)
# save('ridge', 'fc-age', err, trsubs, tsubs, reg2.coef_)
# save('ridge-wf', 'fc-age', err, trsubs, tsubs, reg2.coef_*np.mean(xtr2, axis=0))

# reg3 = Lasso(alpha=1e-3, max_iter=1000).fit(xtr2, ytr)
# yhat3 = reg3.predict(xt2)
# err = rmse(yhat3, yt)
# print(err)
# save('lasso', 'fc-age', err, trsubs, tsubs, reg3.coef_)
# save('lasso-wf', 'fc-age', err, trsubs, tsubs, reg3.coef_*np.mean(xtr2, axis=0))

reg4 = Lasso(alpha=1e-3, max_iter=1000).fit(xtr0, ytr)
yhat4 = reg4.predict(xt0)
err = rmse(yhat4, yt)
print(err)
save('lasso', 'ab5-age', err, trsubs, tsubs, reg4.coef_)
save('lasso-wf', 'ab5-age', err, trsubs, tsubs, reg4.coef_*np.mean(xtr0, axis=0))

reg5 = Lasso(alpha=1e-3, max_iter=1000).fit(xtr1, ytr)
yhat5 = reg5.predict(xt1)
err = rmse(yhat5, yt)
print(err)
save('lasso', 'res1-age', err, trsubs, tsubs, reg5.coef_)
save('lasso-wf', 'res1-age', err, trsubs, tsubs, reg5.coef_*np.mean(xtr1, axis=0))

# yhat = (yhat0+yhat1)/2
# print(rmse(yhat, yt))

yhat = (yhat4+yhat5)/2
print(rmse(yhat, yt))

print(rmse(0, yt))

KeyboardInterrupt: 

In [145]:
def corr(x, y):
    x = x-np.mean(x, axis=0)
    y = y-np.mean(y, axis=0)
    syy = np.mean(y**2)
    sxx = np.mean(x**2, axis=0)
    sxy = np.einsum('nf,n->f', x, y)
    sxy /= x.shape[0]
    return sxy/(sxx*syy)**0.5

rho = corr(xtr2, ytr)
idcs_rho = np.argsort(np.abs(rho))
print(idcs_rho[-10:])

[21733 30686 26584 16845  4398 15416  9995    73  2700 16381]


In [152]:
# Load previously estimated weights

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

n = 50
run = 0

models = ['lasso-wf', 'lasso-wf', 'ridge-wf', 'lasso-wf']
inps = ['ab5-age', 'res1-age', 'fc-age', 'fc-age']
top10 = []

for model,inp in zip(models, inps):
    dct = pickle.load(open(f'{demodir}/weights/{inp}/{model}{run}.pkl', 'rb'))
    idcs = np.argsort(np.abs(dct['w']))
    top10.append(idcs[-n:])
    
e0, e1, e01, e2, e3, e4 = [], [], [], [], [], []
    
x0 = aps5[:,top10[0]]
x1 = res1[:,top10[1]]
x2 = fc[:,top10[2]]
x3 = fc[:,top10[3]]
x4 = fc[:,idcs_rho[-n:]]
y = age

for i in range(20):

    x0tr, x0t, x1tr, x1t, x2tr, x2t, x3tr, x3t, x4tr, x4t, ytr, yt = train_test_split(
        x0, x1, x2, x3, x4, y, train_size=0.5)

    reg = Ridge(alpha=1).fit(x0tr, ytr)
    yhat0 = reg.predict(x0t)
    err = rmse(yhat0, yt)
    print(err)
    e0.append(err)

    reg = Ridge(alpha=1).fit(x1tr, ytr)
    yhat1 = reg.predict(x1t)
    err = rmse(yhat1, yt)
    print(err)
    e1.append(err)

    err = rmse((yhat0+yhat1)/2, yt)
    print(err)
    e01.append(err)

    reg = Ridge(alpha=1).fit(x2tr, ytr)
    yhat2 = reg.predict(x2t)
    err = rmse(yhat2, yt)
    print(err)
    e2.append(err)

    reg = Ridge(alpha=1).fit(x3tr, ytr)
    yhat3 = reg.predict(x3t)
    err = rmse(yhat3, yt)
    print(err)
    e3.append(err)

    reg = Ridge(alpha=1).fit(x4tr, ytr)
    yhat4 = reg.predict(x4t)
    err = rmse(yhat4, yt)
    print(err)
    e4.append(err)

for e in [e0, e1, e01, e2, e3, e4]:
    print(np.mean(e), np.std(e))

2.8340317707949927
2.7206879433720785
2.59642860422112
2.876477324959221
2.81305144173882
3.170012490742997
2.8266013138942396
2.742073707356028
2.608250885350109
2.916399676865828
2.8217569886948652
3.206357941305939
2.819740438900041
2.7329895503849535
2.6069670250090597
2.877266774219085
2.7749205170916276
3.2016632089868047
2.8708982477439164
2.7421592094887886
2.6183190944060106
2.9161845086483535
2.798926841455515
3.1814403325970217
2.8603130753408545
2.7563200548234392
2.6026910837453663
2.904342978145068
2.7940963455142516
3.132598908779738
2.8659610065204775
2.721745783183299
2.6152155184418957
2.940825383752652
2.805119680983553
3.187084099908811
2.8695266486022493
2.7417743328390856
2.61341758374129
2.913048982556593
2.8102634422328463
3.1616399288484405
2.8049641619447567
2.721082066717549
2.5703793329231925
2.83456057211338
2.76832810662134
3.121792655691711
2.9045572318098047
2.72371649247079
2.6318404139894382
2.9095257547806828
2.8208982281355937
3.186190656872867
2.839